# Fase 1: Importar las dependencias


In [1]:
import numpy as np
import math
import re                   #expresiones regulares
import pandas as pd
from bs4 import BeautifulSoup #para trabajar con txt en diferentes formatos


In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
#esto es solamente en GoogleColab, es para cargar la última TF

from tensorflow.keras import layers
import tensorflow_datasets as tfds #de aquí tomaremos el 'tokenizador'

# Fase 2: Pre Procesado de Datos

## Carga de Ficheros

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fuente del dataset: https://www.kaggle.com/itachi9604/disease-symptom-description-dataset?select=dataset.csv

In [4]:
data = pd.read_csv('/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/predicción de enfermedades/dataset.csv')

print(data.shape)

total_nan_values = data.isnull().sum()
print (total_nan_values)

(4920, 18)
Disease          0
Symptom_1        0
Symptom_2        0
Symptom_3        0
Symptom_4      348
Symptom_5     1206
Symptom_6     1986
Symptom_7     2652
Symptom_8     2976
Symptom_9     3228
Symptom_10    3408
Symptom_11    3726
Symptom_12    4176
Symptom_13    4416
Symptom_14    4614
Symptom_15    4680
Symptom_16    4728
Symptom_17    4848
dtype: int64


In [5]:
data.head(5)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.fillna('$$$', inplace=True)

In [7]:
data["sym"] = data["Symptom_1"].astype(str) + data["Symptom_2"].astype(str)+ data["Symptom_3"].astype(str)+ data["Symptom_4"].astype(str)+ data["Symptom_5"].astype(str)+ data["Symptom_6"].astype(str)+ data["Symptom_7"].astype(str)+ data["Symptom_8"].astype(str)+ data["Symptom_9"].astype(str)+ data["Symptom_10"].astype(str)+ data["Symptom_11"].astype(str)+ data["Symptom_12"].astype(str)+ data["Symptom_13"].astype(str)+ data["Symptom_14"].astype(str)+ data["Symptom_15"].astype(str)+ data["Symptom_16"].astype(str)+ data["Symptom_17"].astype(str)

In [8]:
data

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17,sym
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,itching skin_rash nodal_skin_eruptions dischro...
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,skin_rash nodal_skin_eruptions dischromic _pa...
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,itching nodal_skin_eruptions dischromic _patch...
3,Fungal infection,itching,skin_rash,dischromic _patches,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,itching skin_rash dischromic _patches$$$$$$$$$...
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,itching skin_rash nodal_skin_eruptions$$$$$$$$...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,vomiting headache nausea spinning_movements l...
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,skin_rash pus_filled_pimples blackheads scurr...
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,burning_micturition bladder_discomfort foul_s...
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,$$$,skin_rash joint_pain skin_peeling silver_like...


In [9]:
data.drop(['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6', 'Symptom_7','Symptom_8','Symptom_9','Symptom_10','Symptom_11','Symptom_12','Symptom_13',	'Symptom_14',	'Symptom_15',	'Symptom_16',	'Symptom_17'], axis=1, inplace=True)

In [10]:
data.isnull().sum()

Disease    0
sym        0
dtype: int64

In [11]:
data

,Disease,sym
0,Fungal infection,itching skin_rash nodal_skin_eruptions dischro...
1,Fungal infection,skin_rash nodal_skin_eruptions dischromic _pa...
2,Fungal infection,itching nodal_skin_eruptions dischromic _patch...
3,Fungal infection,itching skin_rash dischromic _patches$$$$$$$$$...
4,Fungal infection,itching skin_rash nodal_skin_eruptions$$$$$$$$...
...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting headache nausea spinning_movements l...
4916,Acne,skin_rash pus_filled_pimples blackheads scurr...
4917,Urinary tract infection,burning_micturition bladder_discomfort foul_s...
4918,Psoriasis,skin_rash joint_pain skin_peeling silver_like...


In [33]:
from sklearn.preprocessing import LabelEncoder

In [34]:
test_encoder = LabelEncoder()

In [35]:
data.loc[:, 'Disease'] = test_encoder.fit_transform(data['Disease'])

In [36]:
data.head()

,Disease,sym
0,15,itching skin_rash nodal_skin_eruptions dischro...
1,15,skin_rash nodal_skin_eruptions dischromic _pa...
2,15,itching nodal_skin_eruptions dischromic _patch...
3,15,itching skin_rash dischromic _patches$$$$$$$$$...
4,15,itching skin_rash nodal_skin_eruptions$$$$$$$$...


## Pre Procesado

### Limpieza

In [37]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    
    # Eliminamos la @ y su mención
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) #https://regexr.com/ --> loquizar cualquier cosa que empiece con @+lo que sea y se sutituye por ' '

    # Eliminamos los links de las URLs
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) #el ? significa que el caracter anterior es opcional
    
    # Nos quedamos solamente con los caracteres
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet) #eliminar todo menos va con el ^ y pongo en []
    
    # Eliminamos espacios en blanco adicionales
    tweet = re.sub(r" +", ' ', tweet)
    
    return tweet

In [38]:
sym = [clean_tweet(tweet) for tweet in data.sym]

In [39]:
#disease = [clean_tweet(tweet) for tweet in data.Disease]

In [40]:
set(data.Disease)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40}

### Tokenización

In [41]:
#De una lista de caracteres pasaremos a una lista de números

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    sym, target_vocab_size=2**16 #tamaño del vocabulario, en este casos las 65000 más frecuentes. Las menos frecuentes dificilmente la IA las pueda interpretar.
)

data_inputs = [tokenizer.encode(sentence) for sentence in sym]

### Padding

In [42]:
#Padding --> Agrega CEROS hasta igualar todas las frases en su extención

MAX_LEN = max([len(sentence) for sentence in data_inputs]) #max busca la frase más larga
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

### Dividimos en los conjuntos de entrenamiento y de testing


In [43]:
#Esto es porque los Tweet estan ordenados 800mil neg y 800mil pos, sacaremos 8000 al azar de la primera mitad y 8000 de la segunda mitad

test_idx = np.random.randint(0, 4919, 3000) #los random de los neg

In [44]:
data_labels = data.Disease.values

In [45]:
test_inputs = data_inputs[test_idx] #txt tokenizados y padeado para entrenar la red neuronal
test_labels = data_labels[test_idx] #etiquetas 0 y 1 para entrenar la red neuronal
train_inputs = np.delete(data_inputs, test_idx, axis=0) #lo mismo pero le saco los utilizados arriba
train_labels = np.delete(data_labels, test_idx) #sin axis porque la etiqueta no es una matriz como lo son los txt

In [46]:
print(test_inputs.shape)
print(test_labels.shape)
print(train_inputs.shape)
print(train_labels.shape)

(3000, 52)
(3000,)
(2686, 52)
(2686,)


In [47]:
print(type(test_inputs))
print(type(test_labels))
print(type(train_inputs))
print(type(train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Fase 3: Construción del modelo

In [61]:
#Aquí comienza la CONVULSION, es decir, la aplicación de los filtros

class DCNN(tf.keras.Model): #instanciamos una clase
    
    def __init__(self,
                 vocab_size, #tamaño del vocab, dado por el tokenizador
                 emb_dim=253, #cada palabra será resumida a un numero de 128 caraceteres
                 nb_filters=100, #se pondrán 50 filtros de 4, 3 y 2 palabras
                 FFN_units=1024, #número de neuronas de la capa oculta
                 nb_classes=40, #número de categorias de salida, en este caso son 2 por o neg
                 dropout_rate=0.1, #se usa para el entrenamiento, algunas neuronas se apagan para que no aprendan todas juntas, en cada vuelta el 10% se apagará 
                 training=False, #en la fase de predicción las neuronas funcionan todas
                 name="dcnn"): #le damos un nombre al modelo
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim) #tamaño del vocab
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2, #filtro de 2 palabras
                                    padding="valid",
                                    activation="relu") #cdo se activa la neurona? con esto que anda bien, es automático
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,#filtro de 3 palabras
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4, #filtro de 4 palabras
                                      padding="valid",
                                      activation="relu")
#El pooling comprimirá la info        
        self.pool = layers.GlobalMaxPool1D() # No tenemos variable de entrenamiento
                                             # así que podemos usar la misma capa 
                                             # para cada paso de pooling

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate) #esto es para que no colapse la red
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")#para clasificacion binaria, si hubiera más categorias no podría usarse
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")#para pasar de 2 categorias/binaria a más de dos
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Paso 4: Aplicación

## Configuración

In [62]:
#concretizar todas las variables y parametros que se crearon arriba

VOCAB_SIZE = tokenizer.vocab_size # 65540

EMB_DIM = 200 #cada palabra se identificara con 200 cordenadas
NB_FILTERS = 100 #numero de filtros de la red
FFN_UNITS = 512 #numero de unidades de feet fowards en la capa oculta
NB_CLASSES = 40 #len(set(train_labels)) --> según las etiquetas de entrenamiento

DROPOUT_RATE = 0.2 #20% de las neuronas se apagan en el entrenamiento

BATCH_SIZE = 10 #se probará de 32 en 32 elementos = tweets
NB_EPOCHS = 10 # el data set se iterará 5 veces para mejorar la precisión

## Entrenamiento

In [63]:
#se creea el OBJETO Deep Convutional Neuronal Network
#se le pasa la configuración anterior

Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [64]:
#¿qué haremos cdo el proceso de clasificacion es bianario?

if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

#si no es binario se resuelve de la siguiente manera:

else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [57]:
#Esto es para que si se corta la sesión de GoogleColab y no arrancar desde el principio
#SIRVE PARA TENSOR FLOW

checkpoint_path = "/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/predicción de enfermedades/ckpt"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

Último checkpoint restaurado!!


In [65]:
#Ajustar los datos

Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

Epoch 1/10
269/269 [==============================] - 3s 6ms/step - loss: nan - sparse_categorical_accuracy: 0.0253
Epoch 2/10
269/269 [==============================] - 2s 6ms/step - loss: nan - sparse_categorical_accuracy: 0.0227
Epoch 3/10
269/269 [==============================] - 2s 6ms/step - loss: nan - sparse_categorical_accuracy: 0.0227
Epoch 4/10
269/269 [==============================] - 1s 5ms/step - loss: nan - sparse_categorical_accuracy: 0.0227
Epoch 5/10
269/269 [==============================] - 1s 5ms/step - loss: nan - sparse_categorical_accuracy: 0.0227
Epoch 6/10
269/269 [==============================] - 2s 6ms/step - loss: nan - sparse_categorical_accuracy: 0.0227
Epoch 7/10
269/269 [==============================] - 2s 6ms/step - loss: nan - sparse_categorical_accuracy: 0.0227
Epoch 8/10
269/269 [==============================] - 2s 6ms/step - loss: nan - sparse_categorical_accuracy: 0.0227
Epoch 9/10
269/269 [==============================] - 1s 5ms/step - loss

'/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/predicción de enfermedades/ckpt/ckpt-4'

## Evaluación

In [66]:
#aqui aplica lo entrenado a news que no ha leido antes

results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

300/300 [==============================] - 1s 3ms/step - loss: nan - sparse_categorical_accuracy: 0.0263
[nan, 0.026333333924412727]


Aquí se prueba con sintomaas el entrenamiento anterior

In [67]:
Dcnn(np.array([tokenizer.encode("skin_rash	nodal_skin_eruptions	dischromic _patches")]), training=False).numpy()

#El arreglo tiene las 40 etiquetas, donde estuviese la diferencia sería la enfermedad, pero la CNN no está ajustada lo suficiente

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan]], dtype=float32)